In [29]:
import pandas as pd
data_df=pd.read_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\raw_data_extend\\context\\beam_merge_1.csv",encoding='unicode_escape',header=None)
data_df.columns = ["label_class", "type", "oldname", "newname", "oldStmt", "newStmt",  "edge"]
data_df

,label_class,type,oldname,newname,oldStmt,newStmt,edge
0,1,method,testLateDropping,testLateDroppingOrdered,public void testLateDropping() throws Exceptio...,public void testLateDroppingOrdered() throws E...,"<testLateDropping,org.apache.beam.runners.core..."
1,1,method,getMinTimestamp,getMinimumTimestamp,Instant getMinTimestamp()Optional.empt,Instant getMinimumTimestamp()Optional.empt,"<getMinTimestamp,org.apache.beam.runners.direc..."
2,1,variable,timer,key,protected void fireTimer(InternalTimer<ByteBuf...,public void setCurrentKey(Object key) {},"<timer,beam>|<timer,advanceInputWatermark>|<ti..."
3,1,type,PrintHandler,LoggingHandler,public static class PrintHandler implements Jo...,public static class LoggingHandler implements ...,"<PrintHandler,process>|<PrintHandler,PrintHand..."
4,1,variable,p,pipeline,Pipeline p = TestPipeline.create(options);,Pipeline pipeline = TestPipeline.create(options);,"<p,beam>|<p,testRunStreamingJobNotUsingPAssert..."
5,1,variable,processor1,processor2,BundleProcessor<?> processor1 = sdkHarnessClie...,BundleProcessor processor2 = sdkHarnessClient....,"<processor1,beam>|<processor1,process>|<proces..."
6,1,field,currentRecord,record,private T currentRecord;,"Object record = reader.read(null, decoder);","<currentRecord,beam>|<currentRecord,AvroSource..."
7,1,field,DEFAULT_SHARD_TEMPLATE,DEFAULT_UNWINDOWED_SHARD_TEMPLATE,public static final String DEFAULT_SHARD_TEMPL...,public static final String DEFAULT_UNWINDOWED_...,"<DEFAULT_SHARD_TEMPLATE,beam>|<DEFAULT_SHARD_T..."
8,1,method,setRowOption,setOption,"public Builder setRowOption(String optionName,...","public Builder setOption(String optionName, Fi...","<setRowOption,setOption>|<setRowOption,getAllO..."
9,1,method,bindCombiningValue,bindCombining,"CombiningState<InputT, AccumT, OutputT> bindCo...","CombiningState<InputT, AccumT, OutputT> bindCo...","<bindCombiningValue,bindKeyedCombiningValue>|<..."


In [30]:
def isMatch( s: str, p: str) -> bool:
    m, n = len(s) + 1, len(p) + 1
    dp = [[False] * n for _ in range(m)]
    dp[0][0] = True
    for j in range(2, n, 2):
        dp[0][j] = dp[0][j - 2] and p[j - 1] == '*'
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = dp[i][j - 2] or dp[i - 1][j] and (s[i - 1] == p[j - 2] or p[j - 2] == '.') \
                if p[j - 1] == '*' else \
                dp[i - 1][j - 1] and (p[j - 1] == '.' or s[i - 1] == p[j - 1])
    return dp[-1][-1]
#新增一列记录：把标识符名字去掉
#x：statement
import re  # 第一步，要引入re模块
def delName(x,k_index,isOld):

    classIndex=isMatch(x, '.* *class *.*{.*')
    interfaceIndex = isMatch(x, '.* *interface *.*{.*')
    enumIndex = isMatch(x, '.* *enum *.*{.*')
    assignIndex = isMatch(x, '.* *= *.*{.*')
    index = x.find("{")

    if index != -1 and ((not classIndex) and (not interfaceIndex) and (not enumIndex)and(not assignIndex) ):
        # method
        x = x[index:]
    else:
        if assignIndex:
            equidx = x.find("=")
            if x[0:equidx].find("{") != -1:
                x = x[index:]
        else:
            if isOld:
                name = data_df.iloc[k_index]['oldname']
                # print(name,x)
            else:
                name = data_df.iloc[k_index]['newname']
            x = x.replace(name, "_")
  
    return x

data_df['oldStmt_body'] = data_df["oldStmt"].apply(lambda x: delName(x,data_df.loc[data_df['oldStmt']==x].index[0],True))
data_df['newStmt_body'] = data_df["newStmt"].apply(lambda x: delName(x,data_df.loc[data_df['newStmt']==x].index[0],False))
data_df

,label_class,type,oldname,newname,oldStmt,newStmt,edge,oldStmt_body,newStmt_body
0,1,method,testLateDropping,testLateDroppingOrdered,public void testLateDropping() throws Exceptio...,public void testLateDroppingOrdered() throws E...,"<testLateDropping,org.apache.beam.runners.core...",{MetricsContainerImpl container = new MetricsC...,{testLateDropping(true);}
1,1,method,getMinTimestamp,getMinimumTimestamp,Instant getMinTimestamp()Optional.empt,Instant getMinimumTimestamp()Optional.empt,"<getMinTimestamp,org.apache.beam.runners.direc...",Instant _()Optional.empt,Instant _()Optional.empt
2,1,variable,timer,key,protected void fireTimer(InternalTimer<ByteBuf...,public void setCurrentKey(Object key) {},"<timer,beam>|<timer,advanceInputWatermark>|<ti...",{,{}
3,1,type,PrintHandler,LoggingHandler,public static class PrintHandler implements Jo...,public static class LoggingHandler implements ...,"<PrintHandler,process>|<PrintHandler,PrintHand...",public static class _ implements JobMessagesHa...,public static class _ implements JobMessagesHa...
4,1,variable,p,pipeline,Pipeline p = TestPipeline.create(options);,Pipeline pipeline = TestPipeline.create(options);,"<p,beam>|<p,testRunStreamingJobNotUsingPAssert...",Pi_eline _ = TestPi_eline.create(o_tions);,Pipeline _ = TestPipeline.create(options);
5,1,variable,processor1,processor2,BundleProcessor<?> processor1 = sdkHarnessClie...,BundleProcessor processor2 = sdkHarnessClient....,"<processor1,beam>|<processor1,process>|<proces...",BundleProcessor<?> _ = sdkHarnessClient.getPro...,BundleProcessor _ = sdkHarnessClient.getProces...
6,1,field,currentRecord,record,private T currentRecord;,"Object record = reader.read(null, decoder);","<currentRecord,beam>|<currentRecord,AvroSource...",private T _;,"Object _ = reader.read(null, decoder);"
7,1,field,DEFAULT_SHARD_TEMPLATE,DEFAULT_UNWINDOWED_SHARD_TEMPLATE,public static final String DEFAULT_SHARD_TEMPL...,public static final String DEFAULT_UNWINDOWED_...,"<DEFAULT_SHARD_TEMPLATE,beam>|<DEFAULT_SHARD_T...",public static final String _ = ShardNameTempla...,public static final String _ = ShardNameTempla...
8,1,method,setRowOption,setOption,"public Builder setRowOption(String optionName,...","public Builder setOption(String optionName, Fi...","<setRowOption,setOption>|<setRowOption,getAllO...","{setOption(optionName, FieldType.row(value.get...",{if (value == null) {if (fieldType.getNullable...
9,1,method,bindCombiningValue,bindCombining,"CombiningState<InputT, AccumT, OutputT> bindCo...","CombiningState<InputT, AccumT, OutputT> bindCo...","<bindCombiningValue,bindKeyedCombiningValue>|<...","CombiningState<InputT, AccumT, OutputT> _(Stri...","CombiningState<InputT, AccumT, OutputT> _(Stri..."


In [31]:
print("【same stmt】 \n",data_df.loc[data_df['oldStmt_body'] == data_df['newStmt_body']]['label_class'].value_counts())

【same stmt】 
 1    226
Name: label_class, dtype: int64


In [32]:
print("【Empty method】 \n",data_df.loc[data_df['oldStmt_body']=="{}"]['label_class'].value_counts())
print("【Empty method】 \n",data_df.loc[data_df['newStmt_body']=="{}"]['label_class'].value_counts())
print("【Empty method】 \n",data_df.loc[data_df['oldStmt_body']=="{"]['label_class'].value_counts())
print("【Empty method】 \n",data_df.loc[data_df['newStmt_body']=="{"]['label_class'].value_counts())


【Empty method】 
 Series([], Name: label_class, dtype: int64)
【Empty method】 
 1    1
Name: label_class, dtype: int64
【Empty method】 
 1    24
Name: label_class, dtype: int64
【Empty method】 
 1    24
Name: label_class, dtype: int64


In [33]:
d=data_df.loc[data_df['oldStmt_body'] == data_df['newStmt_body']]
d.to_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\raw_data_extend\\context\\beam_same.csv")

In [82]:
import pandas as pd
# data_df=pd.read_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\raw_data\\test_data_6x\\beam_result_change.csv",encoding='unicode_escape',header=0)
data_df=pd.read_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\raw_data_extend\\changeNum\\dubbo_rawdata_ok_method_.csv",encoding='unicode_escape',header=0)

data_df

,Unnamed: 0,label_class,type,oldname,newname,oldStmt,newStmt,edge,changeNum
0,0,1,method,testInvokeWithBizException,testInvokeWithRuntimeException,public void testInvokeWithBizException(){EasyM...,public void testInvokeWithRuntimeException(){E...,"<testInvokeWithBizException,getInterface>|<tes...",3
1,1,1,method,hasMethodParamter,hasMethodParameter,public boolean hasMethodParamter(String method...,public boolean hasMethodParameter(String metho...,"<hasMethodParamter,getMethodParameter>|<hasMet...",0
2,2,1,method,test_getExtension_WithAutoProxy,test_getExtension_WithWrapper,public void test_getExtension_WithAutoProxy() ...,public void test_getExtension_WithWrapper() th...,"<test_getExtension_WithAutoProxy,getExtensionL...",1
3,3,1,method,test_getExtension_ExceptionNoExtension_NameOnA...,test_getExtension_ExceptionNoExtension_NameOnW...,public void test_getExtension_ExceptionNoExten...,public void test_getExtension_ExceptionNoExten...,<test_getExtension_ExceptionNoExtension_NameOn...,0
4,4,1,method,getRegister,isRegister,public Boolean getRegister(){return register;},public Boolean isRegister(){return register;},"<getRegister,getRegister>|<getRegister,register>|",0
5,5,1,method,getRegister,isRegister,public Boolean getRegister(){return register;},public Boolean isRegister(){return register;},"<getRegister,getRegister>|<getRegister,register>|",0
6,6,1,method,testSystemPropertyOverride,testSystemPropertyOverrideXmlDefault,public void testSystemPropertyOverride() throw...,public void testSystemPropertyOverrideXmlDefau...,"<testSystemPropertyOverride,getApplication>|<t...",2
7,7,1,method,testSystemPropertyOverride,testSystemPropertyOverrideXml,public void testSystemPropertyOverride() throw...,public void testSystemPropertyOverrideXml() th...,"<testSystemPropertyOverride,getApplication>|<t...",2
8,8,1,method,getRoutes,getServiceRoutes,"public Map<String, List<URL>> getRoutes(){retu...","public Map<String, List<URL>> getServiceRoutes...","<getRoutes,routes>|<getRoutes,unmodifiableList...",1
9,9,1,method,tooverwrides,toOverrides,"private Map<String, Map<String, String>> toove...","private Map<String, Map<String, String>> toOve...","<tooverwrides,overwrides>|<tooverwrides,toover...",2


In [83]:

print("原始的类别分布：")
print(data_df['label_class'].value_counts())
print("没有变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] ==0]['label_class'].value_counts())

print("变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] >0]['label_class'].value_counts())

原始的类别分布：
0    1005
1      54
Name: label_class, dtype: int64
没有变化过的类别分布：
0    670
1     24
Name: label_class, dtype: int64
变化过的类别分布：
0    335
1     30
Name: label_class, dtype: int64


In [84]:
def isMatch( s: str, p: str) -> bool:
    m, n = len(s) + 1, len(p) + 1
    dp = [[False] * n for _ in range(m)]
    dp[0][0] = True
    for j in range(2, n, 2):
        dp[0][j] = dp[0][j - 2] and p[j - 1] == '*'
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = dp[i][j - 2] or dp[i - 1][j] and (s[i - 1] == p[j - 2] or p[j - 2] == '.') \
                if p[j - 1] == '*' else \
                dp[i - 1][j - 1] and (p[j - 1] == '.' or s[i - 1] == p[j - 1])
    return dp[-1][-1]
#新增一列记录：把标识符名字去掉
#x：statement
import re  # 第一步，要引入re模块
def delName(x,k_index,isOld):

    classIndex=isMatch(x, '.* *class *.*{.*')
    interfaceIndex = isMatch(x, '.* *interface *.*{.*')
    enumIndex = isMatch(x, '.* *enum *.*{.*')
    assignIndex = isMatch(x, '.* *= *.*{.*')
    index = x.find("{")

    if index != -1 and ((not classIndex) and (not interfaceIndex) and (not enumIndex)and(not assignIndex) ):
        # method
        x = x[index:]
    else:
        if assignIndex:
            equidx = x.find("=")
            if x[0:equidx].find("{") != -1:
                x = x[index:]
        else:
            if isOld:
                name = data_df.iloc[k_index]['oldname']
                # print(name,x)
            else:
                name = data_df.iloc[k_index]['newname']
            x = x.replace(name, "_")
  
    return x

data_df['oldStmt_body'] = data_df["oldStmt"].apply(lambda x: delName(x,data_df.loc[data_df['oldStmt']==x].index[0],True))
data_df['newStmt_body'] = data_df["newStmt"].apply(lambda x: delName(x,data_df.loc[data_df['newStmt']==x].index[0],False))
data_df
print("【same stmt】 \n",data_df.loc[data_df['oldStmt_body'] == data_df['newStmt_body']]['label_class'].value_counts())

【same stmt】 
 0    973
1      6
Name: label_class, dtype: int64


In [80]:
data_df.drop(data_df[data_df['oldStmt_body']=="{}"].index, inplace=True)
data_df.drop(data_df[data_df['newStmt_body']=="{}"].index, inplace=True)
data_df.drop(data_df[data_df['oldStmt_body']=="{"].index, inplace=True)
data_df.drop(data_df[data_df['newStmt_body']=="{"].index, inplace=True)

data_df.drop(data_df[data_df['oldStmt_body']=="{}"].index, inplace=True)
data_df.drop(data_df[data_df['newStmt_body']=="{}"].index, inplace=True)
#查看 相同stmt的label分布，一般全是label=0

data_df.drop(data_df[data_df['oldStmt_body'] == data_df['newStmt_body']].index, inplace=True)
print("原始的类别分布：")
print(data_df['label_class'].value_counts())
print("没有变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] ==0]['label_class'].value_counts())

print("变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] >0]['label_class'].value_counts())

原始的类别分布：
1    44
0    27
Name: label_class, dtype: int64
没有变化过的类别分布：
1    18
0     6
Name: label_class, dtype: int64
变化过的类别分布：
1    26
0    21
Name: label_class, dtype: int64


In [64]:
# print("【same stmt】 \n",data_df.loc[data_df['oldStmt_body'] == data_df['newStmt_body']]['changeNum'].value_counts())
import pandas as pd
proj="dubbo"
data_df=pd.read_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\prepocessed_data_extend\\unmasked_test_data\\"+proj+'_test_method.csv')
data_df
print("原始的类别分布：")
print(data_df['label_class'].value_counts())

原始的类别分布：
0    325
1     26
Name: label_class, dtype: int64


In [65]:

print("没有变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] ==0]['label_class'].value_counts())

print("变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] >0]['label_class'].value_counts())

没有变化过的类别分布：
0    258
1      8
Name: label_class, dtype: int64
变化过的类别分布：
0    67
1    18
Name: label_class, dtype: int64


In [66]:

def findNode(x):
    edges = x.split("|")
    #<x,x>,<x,x>.....
    nodeset = set()
    for e in edges:
        index = e.find(',')
        #实体node
        node = e[index + 1:-1].strip()
        if len(node)>0:
            nodeset.add(node)
    return nodeset
        # if score!=None:
def delName(x,index):
    #假设samestmt就是changeNum=0
    changeNum=data_df.iloc[index]['changeNum']
    newStmt=data_df.iloc[index]['newStmt_body']
    if x==newStmt:
        changeNum=0
        pass
    #又对于old edg和new edge有变化的changeNum+
    else:
        oldedge=data_df.iloc[index]['oldedge']
        newedge=data_df.iloc[index]['newedge']
        
        oldnode=findNode(oldedge)
        newnode=findNode(newedge)
#         print(oldnode)
        diffEnt=0
        for n in newnode:
            if n not in oldnode:
                diffEnt=diffEnt+1
                print(n)
   
    
        if diffEnt>0:
            
            changeNum=diffEnt
  
    return changeNum
data_df['changeNum'] = data_df["oldStmt_body"].apply(lambda x: delName(x,data_df.loc[data_df['oldStmt_body']==x].index[0]))




expected
getCode
assertEquals
fail
expected
isTimeout
fail
RpcException
suffix
parameters
startsWith
endsWith
prefix
keySet
fullKey
Ext5Wrapper2
Ext5Wrapper1
RpcConstants
exporters
addParameterAndEncoded
info
loadMonitor
export
monitorUrl
invoker
providerURL
getParameter
proxyFactory
isInfoEnabled
ref
protocol
getInvoker
exporter
toFullString
service
service
serviceRoutes
serviceRoutes
serviceProviders
serviceConsumers
overrides
overrideMap
overrideUrls
toOverrides
overrideMap
overrideUrls
overrideUrls
subscribe
destroy
toFullString
closed
connected
getHandler
getHandler
disconnected
sent
received
isEvent
getClients
ghostClientMap
referenceClientMap
rmiProxyFactory
rmiProxyFactory


In [67]:
print("原始的类别分布：")
print(data_df['label_class'].value_counts())
print("没有变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] ==0]['label_class'].value_counts())

print("变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] >0]['label_class'].value_counts())

原始的类别分布：
0    325
1     26
Name: label_class, dtype: int64
没有变化过的类别分布：
0    306
1      7
Name: label_class, dtype: int64
变化过的类别分布：
1    19
0    19
Name: label_class, dtype: int64


In [61]:
data_df.to_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\prepocessed_data_extend\\unmasked_test_data\\"+proj+'_test_method_data_1.csv')

In [68]:
print("【not same stmt】 \n",data_df.loc[data_df['oldStmt_body'] != data_df['newStmt_body']]['label_class'].value_counts())

【not same stmt】 
 1    21
0    20
Name: label_class, dtype: int64


In [127]:
d=data_df.loc[data_df['oldStmt_body'] != data_df['newStmt_body']]

In [128]:
# d.to_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\prepocessed_data_extend\\unmasked_test_data\\"+proj+'_test_method_data_2.csv')

In [62]:
data_df.drop(data_df[data_df['oldStmt_body'] == data_df['newStmt_body']].index, inplace=True)
print("没有变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] ==0]['label_class'].value_counts())

print("变化过的类别分布：")
print(data_df.loc[data_df['changeNum'] >0]['label_class'].value_counts())


没有变化过的类别分布：
1    13
0    11
Name: label_class, dtype: int64
变化过的类别分布：
0    187
1    149
Name: label_class, dtype: int64


In [63]:
data_df.to_csv("C:\\project\\IdentifierStyle\\data\\VersionDB\\prepocessed_data_extend\\unmasked_test_data\\"+proj+'_test_method_data_2.csv')